### 2.0 Google Analytics Customer Revenue Prediction

##### Importing packages

In [ ]:
# File system manangement
import os
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')
# Numpy and pandas for data manipulation
import numpy as np
import pandas as pd
# Importing random for random selections
import random
# Json for importing JSON columns
import json as json
# Pandas io json normalizing
from pandas.io.json import json_normalize
# Scipy stats for statistical analysis
import scipy.stats as stats
# sklearn preprocessing for dealing with categorical features
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
# to use for woe binning for features with a large number of categories
import scorecardpy as sc
# Light gradient boost classifier
from lightgbm import LGBMRegressor
# Sklearn Inputing data spliting method
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
# Sklearn importing auc as measurement metric
from sklearn.metrics import mean_squared_error
# Gc memory managment
import gc
# Matplotlib pyplot for plotting
import matplotlib.pyplot as plt
# Seabourne for visualization
import seaborn as sns
sns.set_style('white')

#### Importing data

In [ ]:
# Setting up package to import data converting JSON columns into individual features

# https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields
JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
def load_df(csv_path):
    df = pd.read_csv(csv_path, converters={column: json.loads for column in JSON_COLUMNS}, dtype={'fullVisitorId': 'str'})
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    return df

In [ ]:
# Seeting the input and output directory
os.chdir('C:/Users/Jake Cherrie/Documents/Projects/Gstore-Revenue-Prediction')
# Viewing the contained datasets
os.listdir('C:/Users/Jake Cherrie/Data Sets/Gstore Revenue Prediction')

In [ ]:
# Importing the data
trn_df = load_df('C:/Users/Jake Cherrie/Data Sets/Gstore Revenue Prediction/train.csv')
tst_df = load_df('C:/Users/Jake Cherrie/Data Sets/Gstore Revenue Prediction/test.csv')